In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ETL


## Extract

In [ ]:
df  = pd.read_csv("../IoTProcessed_Data.csv")
print(f"Df shape {df.shape}")
print(f"DF info {df.info()}" )

NameError: name 'pd' is not defined

## Transform

In [3]:
# PASO 1. Validar y convertir el tipo de dato de la columna date
print("\n--- TRATAMIENTO DE LA COLUMNA DATE ---")

#Inputamos valores nulos 
#print("Valores nulos por columna  :\n", df.isnull().sum())




--- TRATAMIENTO DE LA COLUMNA DATE ---


In [4]:
# Inputamos filas sin fechas
df = df.dropna(subset=["date"])

#Transformamos el vector date A tipo de date: datetime64
df["date"] = pd.to_datetime(df["date"], errors="coerce")

#Reordenamos cronologicamente
df= df.sort_values("date")
df.head(10)



,date,tempreature,humidity,water_level,N,P,K,Fan_actuator_OFF,Fan_actuator_ON,Watering_plant_pump_OFF,Watering_plant_pump_ON,Water_pump_actuator_OFF,Water_pump_actuator_ON
4023,2023-11-27 06:26:00,29,79,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
4022,2023-11-27 06:31:00,29,78,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
5823,2023-11-27 06:36:00,28,77,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
5822,2023-11-27 06:41:00,28,75,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
5821,2023-11-27 06:46:00,28,74,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
5820,2023-11-27 06:51:00,28,75,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
5819,2023-11-27 06:56:00,28,74,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
5818,2023-11-27 07:01:00,28,72,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
6999,2023-11-27 07:06:00,27,71,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0
8084,2023-11-27 07:11:00,26,71,100,185,190,160,0.0,1.0,1.0,0.0,1.0,0.0


In [5]:
# PASO 2. Detección y tratamiento de duplicados
print("\n--- DETECCIÓN Y TRATAMIENTO DE DUPLICADOS ---")

# La mejor práctica es definir 'duplicado' como una fila donde TODAS las columnas son iguales
# (indicando una lectura de sensor totalmente repetida)
columnas_clave = list(df.columns) # Usa todas las columnas para una detección estricta


# 1. Detección
# Contamos duplicados sobre TODAS las columnas.
duplicados = df.duplicated(subset=columnas_clave)
print("Duplicados ESTRICTOS encontrados (todas las columnas iguales): ", duplicados.sum())

# 2. Eliminación
if duplicados.sum() > 0:
    # Eliminamos las filas que son repeticiones completas
    df = df.drop_duplicates(subset=columnas_clave, keep='first')
    print("Duplicados eliminados.")
else:
    print("No se encontraron duplicados estrictos para eliminar.")

print("Registros finales:", df.shape)




--- DETECCIÓN Y TRATAMIENTO DE DUPLICADOS ---
Duplicados ESTRICTOS encontrados (todas las columnas iguales):  0
No se encontraron duplicados estrictos para eliminar.
Registros finales: (37920, 13)


In [6]:
#PASO 3. Validación de rangos lógicos (valores imposibles)
"""
Sabemos que:

tempreature entre (0°C y 60°C.)

humidity entre (0% y 100%.)

water_level entre (0% y 100%.)

N, P, K entre (0 y 255.)

Actuadores: 0 o 1.

"""
# --- 1. PASO DE OBSERVACIÓN Y DIAGNÓSTICO (Conteo de Sospechosos) ---
temp_sospechosos = df[(df['tempreature'] == 0) | (df['tempreature'] == 100)]
hum_sospechosos = df[(df['humidity'] == 0) | (df['humidity'] == 100)]
print(f"Filas con Temperatura exactamente 0 °C o 100 °C: {len(temp_sospechosos)}")
print(f"Filas con Humedad exactamente 0% o 100%: {len(hum_sospechosos)}")
print("-" * 40)


# --- 2. VALIDACIÓN DE RANGOS LÓGICOS (Límites Duros) ---
# Primero, aseguramos que ningún valor exceda lo que el sensor puede reportar (0-100 o 0-255)
df_logica_invalida = df[
    ~(
        (df['tempreature'].between(0, 100)) &
        (df['humidity'].between(0, 100)) &
        (df['water_level'].between(0, 100)) &
        (df['N'].between(0, 255)) &
        (df['P'].between(0, 255)) &
        (df['K'].between(0, 255)) &
        # Validamos que los actuadores sean 0 o 1
        (df[['Fan_actuator_OFF', 'Fan_actuator_ON', 'Watering_plant_pump_OFF', 'Watering_plant_pump_ON', 'Water_pump_actuator_OFF', 'Water_pump_actuator_ON']].isin([0.0, 1.0])).all(axis=1)
    )
]

print(f"Filas fuera del Rango Lógico Duro (ej. P > 255): {len(df_logica_invalida)}")
if not df_logica_invalida.empty:
    print("--- Filas con Rango Lógico Ilegal (Ejemplo: P=300) ---")
    print(df_logica_invalida)
print("-" * 40)


# --- 3. VALIDACIÓN DE PLASIBILIDAD ESTRICTA (Límites Biológicos) ---
# Definimos los rangos de plausibilidad basados en el conocimiento del dominio (cultivo IoT)

# Rangos Plausibles Estrictos:
# Temperatura: Entre 10 °C (frio) y 45 °C (calor, pero no letal instantáneo)
TEMP_MIN_ESTRICTA = 10
TEMP_MAX_ESTRICTA = 45

# Humedad: Entre 10% (muy seco) y 95% (muy húmedo, evita saturación 100%)
HUM_MIN_ESTRICTA = 10
HUM_MAX_ESTRICTA = 95

# Nivel de agua: Aunque 0 y 100 son posibles, 0-100% es el rango aceptado.
# Nos enfocamos en T y H que son las más críticas para la plausibilidad del sensor/entorno.


# Identificamos las filas que están fuera de este RANGO ESTRICTO
df_plausibilidad_invalida = df[
    ~(
        (df['tempreature'].between(TEMP_MIN_ESTRICTA, TEMP_MAX_ESTRICTA)) &
        (df['humidity'].between(HUM_MIN_ESTRICTA, HUM_MAX_ESTRICTA))
    )
    # Excluimos las filas que ya fallaron el rango lógico, para ver solo las nuevas
    # .drop(df_logica_invalida.index, errors='ignore') 
]

print(f"--- Filas que Fallan la Plausibilidad Estricta (T fuera de 10-45°C o H fuera de 10-95%) ---")
print(f"Total de Filas No Plausibles: {len(df_plausibilidad_invalida)}")
print(df_plausibilidad_invalida)
print("-" * 40)



df.shape

Filas con Temperatura exactamente 0 °C o 100 °C: 0
Filas con Humedad exactamente 0% o 100%: 1343
----------------------------------------
Filas fuera del Rango Lógico Duro (ej. P > 255): 1
--- Filas con Rango Lógico Ilegal (Ejemplo: P=300) ---
                     date  tempreature  humidity  water_level    N    P    K  \
10891 2023-12-07 05:42:00           23        59           63  184  229  259   

       Fan_actuator_OFF  Fan_actuator_ON  Watering_plant_pump_OFF  \
10891               0.0              1.0                      1.0   

       Watering_plant_pump_ON  Water_pump_actuator_OFF  Water_pump_actuator_ON  
10891                     0.0                      0.0                     1.0  
----------------------------------------
--- Filas que Fallan la Plausibilidad Estricta (T fuera de 10-45°C o H fuera de 10-95%) ---
Total de Filas No Plausibles: 8355
                    date  tempreature  humidity  water_level    N    P    K  \
82   2023-12-03 04:44:00           35         0

(37920, 13)

In [7]:
# --- 4. MANEJO DE DATOS ATÍPICOS Y LIMPIEZA (Técnicas de Tratamiento) ---

df_limpio = df.copy()

# 4.1. Eliminar filas con Rangos Lógicos Ilegales (Error de Registro)
# Si el valor es imposible (ej. P=300), eliminamos la fila completa, pues el registro es defectuoso.
df_limpio.drop(df_logica_invalida.index, inplace=True)
print(f"4.1. Eliminadas {len(df_logica_invalida)} filas con Rango Lógico Ilegal (P > 255).")
print(f"Dimensiones después de la eliminación: {df_limpio.shape}")
print("-" * 40)


# 4.2. Tratar Valores de Saturación como Faltantes (Reemplazo a NaN)
# Identificamos y convertimos los valores exactos 0 y 100 en T y H a NaN (fallos por saturación).
print("4.2. Reemplazando valores de saturación (0, 100) en Temperatura y Humedad por NaN...")

saturacion_t = (df_limpio['tempreature'] == 0) | (df_limpio['tempreature'] == 100)
saturacion_h = (df_limpio['humidity'] == 0) | (df_limpio['humidity'] == 100)

df_limpio.loc[saturacion_t, 'tempreature'] = np.nan
df_limpio.loc[saturacion_h, 'humidity'] = np.nan

nan_t_count = df_limpio['tempreature'].isna().sum()
nan_h_count = df_limpio['humidity'].isna().sum()

print(f"   Total de NaN en 'tempreature': {nan_t_count}")
print(f"   Total de NaN en 'humidity': {nan_h_count} (Simulando los 1343 que encontraste)")
print("-" * 40)


# 4.3. Imputación de Valores Faltantes (Técnica: Mediana)
# La mediana es robusta y adecuada para rellenar los huecos creados.
mediana_temp = df_limpio['tempreature'].median()
mediana_hum = df_limpio['humidity'].median()

# Aplicamos la imputación
df_limpio['tempreature'].fillna(mediana_temp, inplace=True)
df_limpio['humidity'].fillna(mediana_hum, inplace=True)

print(f"4.3. Imputación completada:")
print(f"   'tempreature' llenado con la mediana: {mediana_temp:.2f}")
print(f"   'humidity' llenado con la mediana: {mediana_hum:.2f}")
print("-" * 40)


# 4.4. Resumen y Conteo Final de Outliers (Verificación)
# Revisamos cuántos outliers biológicos NO-SATURACIÓN quedan.

outliers_finales = df_limpio[
    ~(
        (df_limpio['tempreature'].between(TEMP_MIN_ESTRICTA, TEMP_MAX_ESTRICTA)) &
        (df_limpio['humidity'].between(HUM_MIN_ESTRICTA, HUM_MAX_ESTRICTA))
    )
]

print(f"--- PASO 4.4: Outliers Plausibles Finales (No-Saturación): {len(outliers_finales)} ---")
print("Nota: El DataFrame 'df_limpio' ahora contiene los datos limpios y listos para análisis.")
print(f"Dimensiones Finales del DataFrame Limpio: {df_limpio.shape}")
print("-" * 40)
print("df_limpio.head()")
#df_limpio.head()
df = df_limpio.copy()

4.1. Eliminadas 1 filas con Rango Lógico Ilegal (P > 255).
Dimensiones después de la eliminación: (37919, 13)
----------------------------------------
4.2. Reemplazando valores de saturación (0, 100) en Temperatura y Humedad por NaN...
   Total de NaN en 'tempreature': 0
   Total de NaN en 'humidity': 1343 (Simulando los 1343 que encontraste)
----------------------------------------
4.3. Imputación completada:
   'tempreature' llenado con la mediana: 17.00
   'humidity' llenado con la mediana: 59.00
----------------------------------------
--- PASO 4.4: Outliers Plausibles Finales (No-Saturación): 7012 ---
Nota: El DataFrame 'df_limpio' ahora contiene los datos limpios y listos para análisis.
Dimensiones Finales del DataFrame Limpio: (37919, 13)
----------------------------------------
df_limpio.head()


C:\Users\Windows\AppData\Local\Temp\ipykernel_9056\2052416156.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_limpio['tempreature'].fillna(mediana_temp, inplace=True)
C:\Users\Windows\AppData\Local\Temp\ipykernel_9056\2052416156.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

In [8]:
import pandas as pd
import numpy as np

# Su DataFrame de ejemplo (Asuma que 'date' ya es de tipo datetime)
# df = pd.read_csv('su_archivo.csv', parse_dates=['date']) 
# O si ya lo cargó: df['date'] = pd.to_datetime(df['date'])

### ------------------------------------------------------------
### PASO 4. Creación de variables derivadas (Feature Engineering)
### ------------------------------------------------------------

# 4.1. Índice NPK (media normalizada de nutriente)
# Asumiendo que los valores máximos de N, P y K son 255.
print("Creando el Índice NPK...")
df['NPK_index'] = (df['N'] + df['P'] + df['K']) / (3 * 255)

# 4.2. Variables de Tiempo (Granularidad Básica)
print("Creando variables de tiempo básicas (hora, día, mes)...")
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
# 'month' solo tendrá 2 valores, pero se mantiene para consistencia.
df['month'] = df['date'].dt.month

# ------------------------------------------------------------------
# 4.3. SEGMENTACIÓN TEMPORAL AVANZADA PARA ESTUDIOS DE CORTO PLAZO (2 MESES)
# ------------------------------------------------------------------

# 4.3.1. Variables de Ciclo Semanal
print("Añadiendo variables de ciclo semanal (día de la semana, fin de semana)...")
# dayofweek: Lunes=0, Domingo=6. Captura patrones semanales.
df['day_of_week'] = df['date'].dt.dayofweek 
# is_weekend: Variable binaria (0 o 1) para un aprendizaje simple.
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int) 

# 4.3.2. Variable de Tendencia (Día de Secuencia)
# Es la variable más importante para capturar la progresión de los 60 días.
print("Añadiendo variable de tendencia (día de experimento)...")
min_date = df['date'].dt.normalize().min()
df['day_of_experiment'] = (df['date'].dt.normalize() - min_date).dt.days + 1

# 4.3.3. Variable Categórica de Segmento del Día
print("Creando variable de segmento del día (Madrugada, Mañana, Tarde, Noche)...")
def categorize_time(hour):
    if 0 <= hour < 6:
        return 'Madrugada'
    elif 6 <= hour < 12:
        return 'Mañana'
    elif 12 <= hour < 18:
        return 'Tarde'
    else: # 18 <= hour < 24
        return 'Noche'

df['time_segment'] = df['hour'].apply(categorize_time)


# ------------------------------------------------------------------
# 4.4. CODIFICACIÓN (One-Hot Encoding)
# Se aplica a las variables categóricas 'day_of_week' y 'time_segment' 
# para que los modelos de Machine Learning puedan interpretarlas.
# ------------------------------------------------------------------

print("Aplicando One-Hot Encoding a las variables categóricas...")

# Codificación de 'day_of_week' (7 columnas: Lunes, Martes, ...)
df_time_dummies_week = pd.get_dummies(df['day_of_week'], prefix='DOW', drop_first=False) 
df = pd.concat([df, df_time_dummies_week], axis=1)

# Codificación de 'time_segment' (4 columnas: Madrugada, Mañana, Tarde, Noche)
df_time_dummies_seg = pd.get_dummies(df['time_segment'], prefix='TimeSeg', drop_first=False) 
df = pd.concat([df, df_time_dummies_seg], axis=1)

# Opcional: Eliminar las columnas originales que ya fueron codificadas.
df = df.drop(columns=['day_of_week', 'time_segment'])

print("Feature Engineering completado.")
# Se puede eliminar también 'day' y 'month' si se confía solo en 'day_of_experiment' y las variables codificadas.
# df = df.drop(columns=['day', 'month'])

# 4.5. Definición de la variable objetivo (target)
target = 'Water_pump_actuator_ON'

print("\nColumnas finales después del Feature Engineering:")
print(df.columns)

Creando el Índice NPK...
Creando variables de tiempo básicas (hora, día, mes)...
Añadiendo variables de ciclo semanal (día de la semana, fin de semana)...
Añadiendo variable de tendencia (día de experimento)...
Creando variable de segmento del día (Madrugada, Mañana, Tarde, Noche)...
Aplicando One-Hot Encoding a las variables categóricas...
Feature Engineering completado.

Columnas finales después del Feature Engineering:
Index(['date', 'tempreature', 'humidity', 'water_level', 'N', 'P', 'K',
       'Fan_actuator_OFF', 'Fan_actuator_ON', 'Watering_plant_pump_OFF',
       'Watering_plant_pump_ON', 'Water_pump_actuator_OFF',
       'Water_pump_actuator_ON', 'NPK_index', 'hour', 'day', 'month',
       'is_weekend', 'day_of_experiment', 'DOW_0', 'DOW_1', 'DOW_2', 'DOW_3',
       'DOW_4', 'DOW_5', 'DOW_6', 'TimeSeg_Madrugada', 'TimeSeg_Mañana',
       'TimeSeg_Noche', 'TimeSeg_Tarde'],
      dtype='object')


In [10]:
### ------------------------------------------------------------
### PASO 5. Normalizacion de variables numericas
### ------------------------------------------------------------

#Usamos MinMaxScaler (escala[0,1]). Ya que nuestras variables fisicas son heterogeneas
from sklearn.preprocessing import MinMaxScaler
features_to_scale = ['tempreature', 'humidity', 'water_level', 'N', 'P', 'K', 'NPK_index']
scaler = MinMaxScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
df


,date,tempreature,humidity,water_level,N,P,K,Fan_actuator_OFF,Fan_actuator_ON,Watering_plant_pump_OFF,...,DOW_1,DOW_2,DOW_3,DOW_4,DOW_5,DOW_6,TimeSeg_Madrugada,TimeSeg_Mañana,TimeSeg_Noche,TimeSeg_Tarde
4023,2023-11-27 06:26:00,0.684211,0.795918,1.00,0.72549,0.745098,0.627451,0.0,1.0,1.0,...,False,False,False,False,False,False,False,True,False,False
4022,2023-11-27 06:31:00,0.684211,0.785714,1.00,0.72549,0.745098,0.627451,0.0,1.0,1.0,...,False,False,False,False,False,False,False,True,False,False
5823,2023-11-27 06:36:00,0.657895,0.775510,1.00,0.72549,0.745098,0.627451,0.0,1.0,1.0,...,False,False,False,False,False,False,False,True,False,False
5822,2023-11-27 06:41:00,0.657895,0.755102,1.00,0.72549,0.745098,0.627451,0.0,1.0,1.0,...,False,False,False,False,False,False,False,True,False,False
5821,2023-11-27 06:46:00,0.657895,0.744898,1.00,0.72549,0.745098,0.627451,0.0,1.0,1.0,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2024-03-30 05:20:00,0.842105,0.591837,0.38,1.00000,1.000000,1.000000,0.0,1.0,0.0,...,False,False,False,False,True,False,True,False,False,False
382,2024-03-30 05:21:00,0.815789,0.591837,0.00,1.00000,1.000000,1.000000,0.0,1.0,0.0,...,False,False,False,False,True,False,True,False,False,False
747,2024-03-30 05:22:00,0.789474,0.591837,0.00,1.00000,1.000000,1.000000,0.0,1.0,0.0,...,False,False,False,False,True,False,True,False,False,False
748,2024-03-30 05:23:00,0.789474,0.591837,0.00,1.00000,1.000000,1.000000,0.0,1.0,0.0,...,False,False,False,False,True,False,True,False,False,False


## LOAD

In [ ]:
#df.to_csv("iot_agriculture_clean.csv", index=False)
# o en formato eficiente
#df.to_parquet("iot_agriculture_clean.parquet", index=False)